In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
import gpt_2_simple as gpt2
import execjs


# Hyperparameters used in GPT-2 model
model_dir = 'src/generate_model/models'
model_name = '2020041912'
numbers = 1
batch_size = 1


def syntax_check(code):
    checker = execjs.compile("""
        function check(code) {
            try {
                eval(code);
                return true;
            } catch (e) {
                return !(e instanceof SyntaxError);
            }
        }
     """)
    return checker.call("check", code)


def generate(temperature, top_p, top_k, prefix):
    sess = gpt2.start_tf_sess()
    sess = gpt2.reset_session(sess)
    gpt2.load_gpt2(sess, 
                   model_dir=model_dir,
                   model_name=model_name,
                   multi_gpu=True)

    text = gpt2.generate(sess,
                         model_dir=model_dir,
                         model_name=model_name,
                         nsamples=numbers, 
                         batch_size=batch_size, 
                         return_as_list=True, 
                         top_p=top_p, 
                         top_k=top_k,
                         temperature=temperature,
                         prefix=prefix,
                         include_prefix=True)[0]
    
    functions = text.split('//JavascriptTop2000Functions\n')[1:]
    
    for function in functions:
        if syntax_check('var NISLTEST = ' + function):
            return function
    
    return functions[1]